In [14]:
import collections
from collections import Counter
from collections import defaultdict
import numpy as np
import os
import pandas as pd
import regex as re
import spacy
import torch
import random
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForMaskedLM
import string

In [3]:
pd.options.mode.chained_assignment = None
os.environ["TOKENIZERS_PARALLELISM"] = "true"
nlp = spacy.load("en_core_web_lg")
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
device_id = 0 if torch.backends.mps.is_available() else -1

model_name = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)
fill_mask_pipeline = pipeline("fill-mask", model=model_name, device=device_id)

du = pd.read_csv("sep_dec_2022_pol.csv") 
print (len(du))

/var/folders/87/9qwg67f164gck_jk0qq9fkc40000gn/T/ipykernel_96615/1352832593.py:12: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  du = pd.read_csv("sep_dec_2022_pol.csv")


12879236


In [7]:
du = du.dropna(subset = ['comment'])
print (len(du))

12879224


In [ ]:
# get threads with three unique posters
threads_with_3plus_users = du.groupby("thread_num")["stuff"].nunique()
du = du[du["thread_num"].isin(threads_with_3plus_users[threads_with_3plus_users >= 3].index)]
len(du) #12,732,657

12732657

In [ ]:
# drop too much punctuation and bible spam 

def drop_bible_verses(df):
    # i used chatgpt for this man idk a better way to do this? 
    bible_regex = r'\b(?:Genesis|Gen|Exodus|Exod|Leviticus|Lev|Numbers|Num|Deuteronomy|Deut|Joshua|Josh|Judges|Judg|Ruth|1 Samuel|1 Sam|2 Samuel|2 Sam|1 Kings|1 Kgs|2 Kings|2 Kgs|1 Chronicles|1 Chr|2 Chronicles|2 Chr|Ezra|Nehemiah|Neh|Esther|Est|Job|Psalms|Psalm|Ps|Proverbs|Prov|Ecclesiastes|Eccl|Song of Solomon|Song|Isaiah|Isa|Jeremiah|Jer|Lamentations|Lam|Ezekiel|Ezek|Daniel|Dan|Hosea|Hos|Joel|Amos|Obadiah|Obad|Jonah|Micah|Nahum|Habakkuk|Hab|Zephaniah|Zeph|Haggai|Hag|Zechariah|Zech|Malachi|Mal|Matthew|Matt|Mark|Mk|Luke|Lk|John|Jn|Acts|Romans|Rom|1 Corinthians|1 Cor|2 Corinthians|2 Cor|Galatians|Gal|Ephesians|Eph|Philippians|Phil|Colossians|Col|1 Thessalonians|1 Thess|2 Thessalonians|2 Thess|1 Timothy|1 Tim|2 Timothy|2 Tim|Titus|Philemon|Phlm|Hebrews|Heb|James|Jas|1 Peter|1 Pet|2 Peter|2 Pet|1 John|2 John|3 John|Jude|Revelation|Rev) \d+:\d+(-\d+)?\b'
    df_filtered = df[~df["comment"].str.contains(bible_regex, case=True, regex=True, na=False)]
    return df_filtered

du = drop_bible_verses(du)
len(du) #12,732,648

/var/folders/87/9qwg67f164gck_jk0qq9fkc40000gn/T/ipykernel_96615/4192613277.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_filtered = df[~df["comment"].str.contains(bible_regex, case=True, regex=True, na=False)]


12732648

In [ ]:
def drop_repeated_word_spam(df):
    repeated_word_regex = r'(?i)\b(\w+)(?:\s+\1){2,}\b'
    mask = df["comment"].astype(str).str.contains(repeated_word_regex, regex=True, na=False)
    return df.loc[~mask] 

du = drop_repeated_word_spam(du)
len(du) #12,705,745

/var/folders/87/9qwg67f164gck_jk0qq9fkc40000gn/T/ipykernel_96615/303654199.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df["comment"].astype(str).str.contains(repeated_word_regex, regex=True, na=False)


12705745

In [ ]:
def drop_high_punctuation_comments(df, threshold=0.075):
    def punctuation_ratio(text):
        if not isinstance(text, str) or len(text) == 0:
            return 0  
        num_punctuation = sum(1 for char in text if char in string.punctuation)
        total_chars = len(text)
        return num_punctuation / total_chars if total_chars > 0 else 0  
    df["punctuation_ratio"] = df["comment"].astype(str).apply(punctuation_ratio)
    df_filtered = df[df["punctuation_ratio"] <= threshold].drop(columns=["punctuation_ratio"])
    return df_filtered

du = drop_high_punctuation_comments(du)
print(len(du)) #12,021,537

In [ ]:
threads_with_3plus_users = du.groupby("thread_num")["stuff"].nunique()
du = du[du["thread_num"].isin(threads_with_3plus_users[threads_with_3plus_users >= 3].index)]
len(du) #12,008,204

12008204

In [ ]:
du = du.dropna(subset=["comment"])
print (len(du)) #12,008,204
du = du.drop_duplicates(subset = ['comment'])
print (len(du)) #10,993,664

12008204
10993664


In [ ]:
threads_with_3plus_users = du.groupby("thread_num")["stuff"].nunique()
du = du[du["thread_num"].isin(threads_with_3plus_users[threads_with_3plus_users >= 3].index)]
len(du) #10,971,948

10971948

In [20]:
du.to_csv("filter_sep_dec_2022_pol.csv")

In [21]:
def df_to_thread_dict(df):
    thread_dict = {
        thread: thread_df.sort_values(by="timestamp")[["num", "comment"]].to_dict(orient='records')
        for thread, thread_df in df.groupby("thread_num")
    }
    return thread_dict

def precompute_thread_dict(df):
    thread_dict = {}
    for thread, thread_df in df.groupby("thread_num"):
        thread_dict[thread] = thread_df.sort_values(by="timestamp")[["num", "comment"]].to_dict(orient='records')
    return thread_dict

def pretokenize_comments(thread_dict):
    for thread, posts in thread_dict.items():
        for post in posts:
            post['tokenized_comment'] = set(post['comment'].lower().split())  # Store pre-tokenized words
    return thread_dict

karina = pretokenize_comments(precompute_thread_dict(du))

In [22]:
import json

In [23]:
def convert_tokenized_sets_to_lists(thread_dict):
    for posts in thread_dict.values():
        for post in posts:
            if isinstance(post.get('tokenized_comment'), set):
                post['tokenized_comment'] = list(post['tokenized_comment'])
    return thread_dict

# Save to JSON file
with open('thread_dict_2022_sep_dec_pol.json', 'w') as f:
    json.dump(convert_tokenized_sets_to_lists(karina), f)

In [24]:
PUNCTUATION_TABLE = str.maketrans("", "", string.punctuation)

def fast_clean_word(word):
    return word.translate(PUNCTUATION_TABLE).lower().replace("’", "").replace("'", "")

def is_recognized_spacy(word):
    token = nlp(word)[0]  
    return token.has_vector and not token.is_oov  

def has_repeating_letters(word):
    return bool(re.search(r"(.)\1\1", word))  

def is_pure_english(word):
    return bool(re.match(r"^[a-zA-Z]+$", word))  

def collect_filtered_words(thread_dict, min_threads=180):
    word_threads = defaultdict(set)

    for thread, posts in thread_dict.items():
        for post in posts:
            for word in map(fast_clean_word, post['tokenized_comment']):
                if word:
                    word_threads[word].add(thread)

    word_thread_counts = {word: len(threads) for word, threads in word_threads.items()}

    frequent_words = {word: count for word, count in word_thread_counts.items() if count >= min_threads}
    
    filtered_frequent_words = {word: count for word, count in frequent_words.items() if len(word) > 3 and word.isalpha()}
    
    recognized_words = {word: count for word, count in filtered_frequent_words.items() if is_recognized_spacy(word)}
    unrecognized_words = {word: count for word, count in filtered_frequent_words.items() if word not in recognized_words}

    filtered_recognized_words = {word: count for word, count in recognized_words.items() if not has_repeating_letters(word) and is_pure_english(word)}
    filtered_unrecognized_words = {word: count for word, count in unrecognized_words.items() if not has_repeating_letters(word) and is_pure_english(word)}

    return filtered_recognized_words, filtered_unrecognized_words, word_thread_counts

filtered_recognized_words, filtered_unrecognized_words, word_thread_counts = collect_filtered_words(karina, min_threads=180)

In [26]:
len(filtered_recognized_words)

21472

In [27]:
def save_words_to_file(word_set, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for word in sorted(word_set):  # Sort for consistency
            f.write(word + "\n")

save_words_to_file(filtered_recognized_words, "2022_sep_dec_pol_recognized_words.txt")
save_words_to_file(filtered_unrecognized_words, "2022_sep_dec_pol_unrecognized_words.txt")